In [61]:
import cv2 
from math import *
import math
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers
from numpy import loadtxt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import csv
import numpy

def toBlackWhiteBinary(img):
    for i in range(16):
        for j in range(16):
            if img[i][j] != 255:
                img[i][j] = 0
    return img

def makeBinaryImages():                
    for i in range(10):
        path = list(r"C:\Users\billy_000\Documents\UF\Neural Networks\Assign1\Neural-Networks-Assignment-1\Assignment1\images\x.PNG")
        path[104] = str(i)
        path = "".join(path)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = toBlackWhiteBinary(img)
        cv2.imwrite(path, img)
        
def ImageToArray(img):
    final = []
    for i in range(16):
        for j in range(16):
            final.append(img[i][j])
            
    for i in range(len(final)):
        if final[i] == 255:
            final[i] = 1
        else:
            final[i] = 0
    return final

def compareImage(Original, Output):
    mismatch = 0
    for i in range(len(Original)):
        if Output[i] != Original[i]:
            mismatch += 1
    return mismatch / len(Original)

def DataSetOfAllImages():
    dataset = []
    for i in range(10):
        path = list(r"C:\Users\billy_000\Documents\UF\Neural Networks\Assign1\Neural-Networks-Assignment-1\Assignment1\images\x.PNG")
        path[104] = str(i)
        path = "".join(path)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        dataset.append(ImageToArray(img))
    return dataset

def Threshold(Arr):
    newArr = []
    for i in Arr:
        if i > 0.5:
            newArr.append(1)
        else:
            newArr.append(0)
    return newArr

def ThresholdResultArrays(arrs):
    newArr = []
    for i in arrs:
        newArr.append(Threshold(i))
    return newArr

def CountWhitePixels(img):
    count = 0
    for i in img:
        if i == 0:
            count += 1
    return count

def ComputeFh(correctImage, Output):
    TotalBlackPixels = 0
    TotalCorrectPixels = 0
    
    for i in range(len(correctImage)):
        if correctImage[i] == 1:
            TotalBlackPixels += 1
            if(Output[i] == 1):
                TotalCorrectPixels += 1
    return TotalCorrectPixels / TotalBlackPixels

def ComputeFfa(correctImage, Output):
    TotalWrongPixels = 0
    
    for i in range(len(correctImage)):
        if (Output[i] == 1) and (correctImage[i] != 1):
            TotalWrongPixels += 1
    
    return TotalWrongPixels / CountWhitePixels(correctImage)
        
        
X = numpy.array(DataSetOfAllImages())
y = X
    
def Model():
    
    model = Sequential()
    model.add(Dense(256, input_dim = 256, activation='relu'))
    #model.add(Dense(256, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    
    model.fit(X, y, epochs=10, batch_size=2, verbose = True)

    
    
    #print(X[1])
    Y = ThresholdResultArrays(model.predict(X))
    
    
    print(compareImage(X[1], Y[1]))
    print(ComputeFh(X[1], Y[1]))
    print(ComputeFfa(X[1], Y[1]))
    
    print(Y)
    

Model()








Epoch 1/10
5/5 [==============================] - 0s 0s/step - loss: 5.4642
Epoch 2/10
5/5 [==============================] - 0s 4ms/step - loss: 4.4395
Epoch 3/10
5/5 [==============================] - 0s 0s/step - loss: 3.9966
Epoch 4/10
5/5 [==============================] - 0s 4ms/step - loss: 3.8286
Epoch 5/10
5/5 [==============================] - 0s 4ms/step - loss: 3.7671
Epoch 6/10
5/5 [==============================] - 0s 4ms/step - loss: 3.7314
Epoch 7/10
5/5 [==============================] - 0s 4ms/step - loss: 3.7231
Epoch 8/10
5/5 [==============================] - 0s 4ms/step - loss: 3.7070
Epoch 9/10
5/5 [==============================] - 0s 4ms/step - loss: 3.7083
Epoch 10/10
5/5 [==============================] - 0s 0s/step - loss: 3.7043
0.27734375
0.6451612903225806
0.07142857142857142
[[1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 